In [68]:
import string
import numpy as np
import pandas as pd
import re
import pickle as pkl

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, WordNetLemmatizer, PorterStemmer, pos_tag, RegexpParser
nltk.download('averaged_perceptron_tagger')
wn = WordNetLemmatizer()
ps = PorterStemmer()
stopwords = stopwords.words('english')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Intel PC G5900\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [41]:
def cleantxt(text):
    text = text.replace('\r', ' ')
    text = text.replace('\n', ' ')
    text = text.replace('\t', ' ')
    text = text.strip()
    text = ''.join([wrd.lower() for wrd in text if wrd not in string.punctuation])
    tokens = re.split('\W+', text)
    tokens = [wrd for wrd in tokens if wrd not in stopwords]
    tokens = [wn.lemmatize(wrd) for wrd in tokens]
    return tokens

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [40]:
df = pd.read_csv("../files/preprocessed_articles.csv")

In [34]:
sents = list(df["text"])

In [37]:
sent_tokens = [cleantxt(sent) for sent in sents]

In [46]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(sent_tokens)]


[TaggedDocument(words=['world', 'news', 'brief', 'sandstorm', 'alert', 'albinism', 'climate', 'change', 'right', 'peru', '19', 'october', '2023', 'climate', 'environment', 'hundred', 'killed', 'injured', 'un', 'world', 'meteorological', 'organization', 'wmo', 'said', 'new', 'report', 'thursday', 'every', 'year', 'around', '2000', 'million', 'tonne', 'dust', 'enter', 'atmosphere', 'darkening', 'sky', 'harming', 'air', 'quality', 'region', 'thousand', 'kilometre', 'away', ''], tags=[0]),
 TaggedDocument(words=['world', 'news', 'brief', 'sandstorm', 'alert', 'albinism', 'climate', 'change', 'right', 'peru', '19', 'october', '2023', 'climate', 'environment', 'hundred', 'killed', 'injured', 'wmo', 'chief', 'petteri', 'taalas', 'said', 'partly', 'due', 'poor', 'water', 'land', 'management', 'phenomenon', 'also', 'exacerbated', 'higher', 'temperature', 'drought', 'brought', 'warming', 'climate', 'leading', 'higher', 'evaporation', 'drier', 'soil'], tags=[1]),
 TaggedDocument(words=['world', '

In [49]:
## Train doc2vec model
model = Doc2Vec(tagged_data,
                 vector_size = 20,
                 window = 2,
                 min_count = 1,
                 epochs = 100)

'''
vector_size = Dimensionality of the feature vectors.
window = The maximum distance between the current and predicted word within a sentence.
min_count = Ignores all words with total frequency lower than this.
alpha = The initial learning rate.
'''


'\nvector_size = Dimensionality of the feature vectors.\nwindow = The maximum distance between the current and predicted word within a sentence.\nmin_count = Ignores all words with total frequency lower than this.\nalpha = The initial learning rate.\n'

In [62]:
embeddings_sent2vec = model.wv.vectors
embeddings_rows = model.wv.vectors.shape[0]
embeddings_cols = model.wv.vectors.shape[1]

In [66]:
with open(f'../files/embeddings_sent2vec_{embeddings_rows}_{embeddings_cols}.pkl', 'wb') as f:
    pkl.dump(model, f)

In [73]:
new_sent = "Give me information about global warming in 2020"
test_doc = cleantxt(new_sent)
test_doc_vector = model.infer_vector(test_doc)
text_ind = model.dv.most_similar(positive = [test_doc_vector])[0][0]

In [65]:
df['text'][13721]

'New UN Global Climate report ‘another strong wake-up call’ over global warming: Guterres 28 March 2019 Climate and Environment ‘Don’t come with a speech, come with a plan’ \x07 Paris Agreement increasingly under threat – WMO chief \x07 Idai victims ‘personify why we need to limit climate change’ \x07 2019 so far: record warmth in Europe, unusual cold in North America \x07 General Assembly meets on climate change and sustainable development  “We need to act, and to act now. The numbers and data are extremely worrisome…We are capable, we have the science, we have the knowledge, we have the tools in hand” to push back on global warming, she added.  '

In [57]:
sents[13721]

'New UN Global Climate report ‘another strong wake-up call’ over global warming: Guterres 28 March 2019 Climate and Environment ‘Don’t come with a speech, come with a plan’ \x07 Paris Agreement increasingly under threat – WMO chief \x07 Idai victims ‘personify why we need to limit climate change’ \x07 2019 so far: record warmth in Europe, unusual cold in North America \x07 General Assembly meets on climate change and sustainable development  “We need to act, and to act now. The numbers and data are extremely worrisome…We are capable, we have the science, we have the knowledge, we have the tools in hand” to push back on global warming, she added.  '